In [88]:
import yfinance as yf

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

In [89]:
POSITIONS = [
    {
        'name': 'Ivan',
        'ticker': 'VST',
        'exchange':'NYSE',
        'weight': 0.4
    },
    {
        'name': 'Ivan',
        'ticker': 'EXPE',
        'exchange':'NASDAQ',
        'weight': 0.3
    },
    {
        'name': 'Ivan',
        'ticker': 'DGX',
        'exchange':'NYSE',
        'weight': 0.3
    },
    {
        'name': 'Abhi',
        'ticker': 'ASR',
        'exchange':'NYSE',
        'weight': 0.4
    },
    {
        'name': 'Abhi',
        'ticker': '0700',  #0700.HK, listed on Listed on HKG (Hong Kong Stock Exchange), traded in HKD
        'exchange':'HKG',
        'weight': 0.25
    },
    {
        'name': 'Abhi',
        'ticker': 'SMCI',
        'exchange':'NASDAQ',
        'weight': 0.35
    },
    {
        'name': 'Conor',
        'ticker': 'VLA', #VLA.PA, listed on Listed on EPA (Euronext Paris), traded in EUR
        'exchange':'EPA',
        'weight': 0.2
    },
    {
        'name': 'Conor',
        'ticker': 'RR', #RR.L,  Listed on LON (London Stock Exchange), traded in GBP
        'exchange':'LON',
        'weight': 0.2
    },
    {
        'name': 'Conor',
        'ticker': 'CTC', #CTC-A.TO, Listed on TSE (Toronto Stock Exchange), traded in CAD
        'exchange':'TSE',
        'weight': 0.2
    },
     {
        'name': 'Conor',
        'ticker': 'SHEL',
        'exchange':'NYSE',
        'weight': 0.2
    },
     {
        'name': 'Conor',
        'ticker': 'CRWD',
        'exchange':'NASDAQ',
        'weight': 0.2
    },
    {
        'name': 'Diarbhail',
        'ticker': 'FLUT',
        'exchange':'NYSE',
        'weight': 0.34
    },
    {
        'name': 'Diarbhail',
        'ticker': 'NVDA',
        'exchange':'NASDAQ',
        'weight': 0.33
    },
    {
        'name': 'Diarbhail',
        'ticker': 'SCAN', #SCAN.V, Listed on TSXV (Toronto Stock Exchange), traded in CAD
        'exchange':'CVE',
        'weight': 0.33
    },
    {
        'name': 'Radu',
        'ticker': 'MP', 
        'exchange':'NYSE',
        'weight': 0.1
    },
    {
        'name': 'Radu',
        'ticker': 'DBK', 
        'exchange':'XETRA',
        'weight': 0.5
    },
    {
        'name': 'Radu',
        'ticker': 'RTX', 
        'exchange':'NYSE',
        'weight': 0.2
    },
    {
        'name': 'Radu',
        'ticker': 'RHM', 
        'exchange':'XETRA',
        'weight': 0.2
    },
    {
        'name': 'Silvia',
        'ticker': 'NVO', 
        'exchange':'NYSE',
        'weight': 0.33
    },
    {
        'name': 'Silvia',
        'ticker': 'NVDA', 
        'exchange':'NASDAQ',
        'weight': 0.33
    },
    {
        'name': 'Silvia',
        'ticker': 'OPRA', 
        'exchange':'NASDAQ',
        'weight': 0.33
    },
]

In [90]:
# Define the start date for data download (from 2000)

FIXED_DATE = '2025-02-21'
INITIAL_VALUE_USD = 1e5

START_DATE = '2000-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')

In [91]:
# check one ticker in Yahoo Finance
ticker = 'SCAN.V'
df_test = yf.download(ticker, start=START_DATE, end=END_DATE)

df_test.tail(5)

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,SCAN.V,SCAN.V,SCAN.V,SCAN.V,SCAN.V
Date,,,,,
2025-02-24,1.89,2.15,1.84,2.05,197541
2025-02-25,1.98,1.99,1.80,1.98,94207
2025-02-26,1.97,2.01,1.91,1.97,191149
2025-02-27,1.97,1.99,1.87,1.98,99640
2025-02-28,1.64,1.70,1.53,1.70,203504


In [92]:
# Get unique tickers and track which ones are from Hong Kong
unique_tickers = []
hk_tickers = []  # Hong Kong stocks (HKD)
eur_tickers = []  # European stocks (EUR)
gbp_tickers = []  # UK stocks (GBP)
cad_tickers = []  # Canadian stocks (CAD)

for position in POSITIONS:
    ticker = position['ticker']
    exchange = position['exchange']
    
    # Format ticker based on exchange
    if exchange == 'NYSE' or exchange == 'NASDAQ':
        formatted_ticker = ticker
    elif exchange == 'HKG':
        formatted_ticker = f"{ticker}.HK"  # Hong Kong stocks use .HK suffix
        hk_tickers.append(formatted_ticker)
    elif exchange == 'EPA':
        formatted_ticker = f"{ticker}.PA"  # Paris stocks use .PA suffix
        eur_tickers.append(formatted_ticker)
    elif exchange == 'LON':
        formatted_ticker = f"{ticker}.L"  # London stocks use .L suffix
        gbp_tickers.append(formatted_ticker)
    elif exchange == 'TSE':
        formatted_ticker = f"{ticker}-A.TO"  # Toronto stocks use .TO suffix (CTC-A is a special case)
        cad_tickers.append(formatted_ticker)
    elif exchange == 'CVE':
        formatted_ticker = f"{ticker}.V"  # .V according to the Yahoo Finance
        cad_tickers.append(formatted_ticker)
    elif exchange == 'XETRA':
        formatted_ticker = f"{ticker}.DE"  # .DE according to the Yahoo Finance (ETR/XETRA stock exchange)
        eur_tickers.append(formatted_ticker)
    else:
        formatted_ticker = ticker
        
    unique_tickers.append(formatted_ticker)

print(f"Downloading data for {len(unique_tickers)} unique tickers from {START_DATE} to {END_DATE}")

# Download historical data for all tickers
stock_data = yf.download(unique_tickers, start=START_DATE, end=END_DATE)

# We'll use the 'Close' prices for our calculations
prices_df = stock_data['Close'].copy()  # Create a copy to avoid SettingWithCopyWarning

# Download HKD to USD exchange rate data if we have Hong Kong stocks
if hk_tickers:
    print("\nDownloading HKD/USD exchange rate data for currency conversion...")
    # HKDUSD=X is the Yahoo Finance ticker for HKD to USD exchange rate
    exchange_rate_data = yf.download('HKDUSD=X', start=START_DATE, end=END_DATE)
    exchange_rate = exchange_rate_data['Close']
    
    # Convert Hong Kong stock prices from HKD to USD using the join approach
    for ticker in hk_tickers:
        print(f"\nProcessing {ticker} using join operation...")
        
        # Check if ticker exists in prices_df
        if ticker in prices_df.columns:
            # Get the HKD prices
            hkd_prices = prices_df[ticker]
        else:
            # Download the data directly
            print(f"Ticker {ticker} not found in original data, downloading directly...")
            single_stock_data = yf.download(ticker, start=START_DATE, end=END_DATE)
            
            if single_stock_data.empty:
                print(f"Error: Could not download data for {ticker}")
                continue
                
            hkd_prices = single_stock_data['Close']
        
        # Create a temporary DataFrame with both series properly indexed
        temp_df = pd.DataFrame({'HKD_Price': hkd_prices})
        temp_df['USD_Rate'] = exchange_rate  # This will align indices automatically
        
        # Calculate USD prices only where both values exist
        temp_df['USD_Price'] = temp_df['HKD_Price'] * temp_df['USD_Rate']
        
        # Print statistics
        print(f"Original HKD prices: {hkd_prices.notna().sum()} non-null values")
        print(f"Exchange rates: {exchange_rate.notna().sum()} non-null values")
        print(f"Converted USD prices: {temp_df['USD_Price'].notna().sum()} non-null values")
        
        # Add the converted prices to prices_df
        prices_df.loc[:, ticker] = temp_df['USD_Price']
        
        print(f"Successfully processed {ticker}")

# Display the first few rows of the data
print("\nFirst few rows of price data (all in USD):")
prices_df.tail(10)

[*****                 10%                       ]  2 of 20 completed

[*********************100%***********************]  20 of 20 completed
[*********************100%***********************]  1 of 1 completed




Processing 0700.HK using join operation...
Original HKD prices: 5114 non-null values
Exchange rates: Ticker
HKDUSD=X    5973
dtype: int64 non-null values
Converted USD prices: 5087 non-null values
Successfully processed 0700.HK

First few rows of price data (all in USD):


Ticker,0700.HK,ASR,CRWD,CTC-A.TO,DBK.DE,DGX,EXPE,FLUT,MP,NVDA,NVO,OPRA,RHM.DE,RR.L,RTX,SCAN.V,SHEL,SMCI,VLA.PA,VST
Date,,,,,,,,,,,,,,,,,,,,
2025-02-17,63.422555,NaN,NaN,NaN,19.691999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,931.599976,635.400024,NaN,NaN,NaN,NaN,3.390,NaN
2025-02-18,64.716226,286.190002,455.359985,140.000000,19.969999,169.649994,204.210007,294.279999,23.450001,139.399994,82.330002,21.850000,928.799988,644.400024,123.136795,1.47,67.820000,55.799999,3.598,169.279999
2025-02-19,64.030661,283.880005,450.140015,145.089996,19.486000,171.750000,206.520004,291.750000,22.990000,139.229996,83.400002,21.360001,932.799988,642.599976,124.778488,1.91,67.599998,60.250000,3.650,169.350006
2025-02-20,62.588232,287.190002,436.140015,144.919998,19.370001,172.139999,205.139999,287.739990,23.180000,140.110001,83.739998,20.340000,889.599976,618.400024,124.479996,2.20,68.379997,59.270000,3.776,163.179993
2025-02-21,66.489875,273.510010,406.679993,141.809998,19.642000,172.240005,196.809998,272.609985,24.299999,134.429993,88.080002,19.590000,894.200012,611.400024,123.290001,2.00,66.699997,56.070000,3.708,150.440002
2025-02-24,63.993577,270.399994,391.720001,142.429993,19.719999,174.059998,199.970001,270.260010,24.270000,130.279999,90.589996,18.510000,951.400024,606.200012,125.430000,1.89,66.889999,51.610001,3.854,142.759995
2025-02-25,62.362602,269.890015,379.750000,146.000000,20.105000,175.410004,199.759995,260.489990,23.559999,126.629997,90.949997,18.150000,952.799988,615.000000,127.339996,1.98,66.900002,45.540001,3.650,137.919998
2025-02-26,64.516381,273.869995,394.559998,146.080002,20.900000,173.460007,199.539993,273.220001,23.770000,131.279999,89.550003,18.360001,970.000000,631.000000,127.309998,1.97,67.400002,51.110001,3.650,148.190002
2025-02-27,63.733273,264.859985,380.239990,143.429993,20.500000,171.550003,193.550003,271.769989,23.459999,120.150002,89.220001,17.980000,1001.000000,731.599976,130.520004,1.97,67.269997,42.950001,3.562,130.009995


In [93]:
# OTHER CURRENCIES : EUR, GBP, CAD
# -------------------------------
# Download EUR to USD exchange rate data if we have European stocks
if eur_tickers:
    print("\nDownloading EUR/USD exchange rate data for currency conversion...")
    eur_usd_data = yf.download('EURUSD=X', start=START_DATE, end=END_DATE)
    eur_usd_rate = eur_usd_data['Close']
    print(f"Downloaded {len(eur_usd_rate)} days of EUR/USD rates")

# Download GBP to USD exchange rate data if we have UK stocks
if gbp_tickers:
    print("\nDownloading GBP/USD exchange rate data for currency conversion...")
    gbp_usd_data = yf.download('GBPUSD=X', start=START_DATE, end=END_DATE)
    gbp_usd_rate = gbp_usd_data['Close']
    print(f"Downloaded {len(gbp_usd_rate)} days of GBP/USD rates")

# Download CAD to USD exchange rate data if we have Canadian stocks
if cad_tickers:
    print("\nDownloading CAD/USD exchange rate data for currency conversion...")
    cad_usd_data = yf.download('CADUSD=X', start=START_DATE, end=END_DATE)
    cad_usd_rate = cad_usd_data['Close']
    print(f"Downloaded {len(cad_usd_rate)} days of CAD/USD rates")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Downloaded 5515 days of EUR/USD rates

Downloaded 5527 days of GBP/USD rates

Downloaded 5583 days of CAD/USD rates


In [94]:
# Convert EUR stocks to USD using the join approach
for ticker in eur_tickers:
    print(f"\nProcessing {ticker} (EUR) using join operation...")
    
    # Check if ticker exists in prices_df
    if ticker in prices_df.columns:
        eur_prices = prices_df[ticker]
    else:
        # Download the data directly
        print(f"Ticker {ticker} not found in original data, downloading directly...")
        stock_data = yf.download(ticker, start=START_DATE, end=END_DATE)
        
        if stock_data.empty:
            print(f"Error: Could not download data for {ticker}")
            continue
            
        eur_prices = stock_data['Close']
    
    # Create a temporary DataFrame with both series properly indexed
    temp_df = pd.DataFrame({'EUR_Price': eur_prices})
    temp_df['USD_Rate'] = eur_usd_rate  # This will align indices automatically
    
    # Calculate USD prices only where both values exist
    temp_df['USD_Price'] = temp_df['EUR_Price'] * temp_df['USD_Rate']
    
    # Print statistics
    print(f"Original EUR prices: {eur_prices.notna().sum()} non-null values")
    print(f"Exchange rates: {eur_usd_rate.notna().sum()} non-null values")
    print(f"Converted USD prices: {temp_df['USD_Price'].notna().sum()} non-null values")
    
    # Add the converted prices to prices_df
    prices_df.loc[:, ticker] = temp_df['USD_Price']
    
    print(f"Successfully processed {ticker}")

# Convert GBP stocks to USD using the join approach
for ticker in gbp_tickers:
    print(f"\nProcessing {ticker} (GBP) using join operation...")
    
    # Check if ticker exists in prices_df
    if ticker in prices_df.columns:
        gbp_prices = prices_df[ticker]
    else:
        # Download the data directly
        print(f"Ticker {ticker} not found in original data, downloading directly...")
        stock_data = yf.download(ticker, start=START_DATE, end=END_DATE)
        
        if stock_data.empty:
            print(f"Error: Could not download data for {ticker}")
            continue
            
        gbp_prices = stock_data['Close']
    
    # Create a temporary DataFrame with both series properly indexed
    temp_df = pd.DataFrame({'GBP_Price': gbp_prices})
    temp_df['USD_Rate'] = gbp_usd_rate  # This will align indices automatically
    
    # Calculate USD prices only where both values exist
    temp_df['USD_Price'] = temp_df['GBP_Price'] * temp_df['USD_Rate']
    
    # Print statistics
    print(f"Original GBP prices: {gbp_prices.notna().sum()} non-null values")
    print(f"Exchange rates: {gbp_usd_rate.notna().sum()} non-null values")
    print(f"Converted USD prices: {temp_df['USD_Price'].notna().sum()} non-null values")
    
    # Add the converted prices to prices_df
    prices_df.loc[:, ticker] = temp_df['USD_Price']
    
    print(f"Successfully processed {ticker}")

# Convert CAD stocks to USD using the join approach
for ticker in cad_tickers:
    print(f"\nProcessing {ticker} (CAD) using join operation...")
    
    # Check if ticker exists in prices_df
    if ticker in prices_df.columns:
        cad_prices = prices_df[ticker]
    else:
        # Download the data directly
        print(f"Ticker {ticker} not found in original data, downloading directly...")
        stock_data = yf.download(ticker, start=START_DATE, end=END_DATE)
        
        if stock_data.empty:
            print(f"Error: Could not download data for {ticker}")
            continue
            
        cad_prices = stock_data['Close']
    
    # Create a temporary DataFrame with both series properly indexed
    temp_df = pd.DataFrame({'CAD_Price': cad_prices})
    temp_df['USD_Rate'] = cad_usd_rate  # This will align indices automatically
    
    # Calculate USD prices only where both values exist
    temp_df['USD_Price'] = temp_df['CAD_Price'] * temp_df['USD_Rate']
    
    # Print statistics
    print(f"Original CAD prices: {cad_prices.notna().sum()} non-null values")
    print(f"Exchange rates: {cad_usd_rate.notna().sum()} non-null values")
    print(f"Converted USD prices: {temp_df['USD_Price'].notna().sum()} non-null values")
    
    # Add the converted prices to prices_df
    prices_df.loc[:, ticker] = temp_df['USD_Price']
    
    print(f"Successfully processed {ticker}")

# --------------------------------------------


Processing VLA.PA (EUR) using join operation...
Original EUR prices: 4523 non-null values
Exchange rates: Ticker
EURUSD=X    5515
dtype: int64 non-null values
Converted USD prices: 4497 non-null values
Successfully processed VLA.PA

Processing DBK.DE (EUR) using join operation...
Original EUR prices: 6433 non-null values
Exchange rates: Ticker
EURUSD=X    5515
dtype: int64 non-null values
Converted USD prices: 5385 non-null values
Successfully processed DBK.DE

Processing RHM.DE (EUR) using join operation...
Original EUR prices: 6433 non-null values
Exchange rates: Ticker
EURUSD=X    5515
dtype: int64 non-null values
Converted USD prices: 5385 non-null values
Successfully processed RHM.DE

Processing RR.L (GBP) using join operation...
Original GBP prices: 6407 non-null values
Exchange rates: Ticker
GBPUSD=X    5527
dtype: int64 non-null values
Converted USD prices: 5370 non-null values
Successfully processed RR.L

Processing CTC-A.TO (CAD) using join operation...
Original CAD prices: 

In [95]:
# Check that each person's weights sum to 100%
person_weights = {}

for position in POSITIONS:
    name = position['name']
    weight = position['weight']
    
    if name not in person_weights:
        person_weights[name] = 0
    
    person_weights[name] += weight

# Create a DataFrame for visualization
weights_df = pd.DataFrame({
    'Person': list(person_weights.keys()),
    'Total Weight': list(person_weights.values())
})

# Add a status column
weights_df['Status'] = weights_df['Total Weight'].apply(lambda x: 'OK' if abs(x - 1.0) < 0.01 else 'ERROR - Not 100%')

# Display the results
print("Weight sum for each person:")
for name, total_weight in person_weights.items():
    print(f"{name}: {total_weight:.2f} ({'OK' if abs(total_weight - 1.0) < 0.01 else 'ERROR - Not 100%'})")

# Create a bar chart to visualize the weights
fig = px.bar(
    weights_df, 
    x='Person', 
    y='Total Weight',
    color='Status',
    title='Portfolio Weight Distribution by Person',
    text='Total Weight',
    color_discrete_map={'OK': 'green', 'ERROR - Not 100%': 'red'}
)

# Add a horizontal line at 1.0 (100%)
fig.add_shape(
    type="line",
    x0=-0.5,
    y0=1.0,
    x1=len(weights_df)-0.5,
    y1=1.0,
    line=dict(color="black", width=2, dash="dash")
)

# Format the y-axis as percentage
fig.update_layout(
    yaxis=dict(title="Total Weight", tickformat=".0%"),
    template='presentation'
)

# Show the figure
fig.show()

Weight sum for each person:
Ivan: 1.00 (OK)
Abhi: 1.00 (OK)
Conor: 1.00 (OK)
Diarbhail: 1.00 (OK)
Radu: 1.00 (OK)
Silvia: 0.99 (ERROR - Not 100%)


In [96]:
# Allocate the initial investment based on weights
portfolio_allocations = []

for position in POSITIONS:
    name = position['name']
    ticker = position['ticker']
    exchange = position['exchange']
    weight = position['weight']
    
    # Format ticker based on exchange
    if exchange == 'NYSE' or exchange == 'NASDAQ':
        formatted_ticker = ticker
    elif exchange == 'HKG':
        formatted_ticker = f"{ticker}.HK"  # Hong Kong stocks use .HK suffix
    elif exchange == 'EPA':
        formatted_ticker = f"{ticker}.PA"  # Paris stocks use .PA suffix
    elif exchange == 'LON':
        formatted_ticker = f"{ticker}.L"  # London stocks use .L suffix
    elif exchange == 'TSE':
        formatted_ticker = f"{ticker}-A.TO"  # Toronto stocks use .TO suffix (CTC-A is a special case)
    elif exchange == 'CVE':
        formatted_ticker = f"{ticker}.V"  # .V according to the Yahoo Finance
    elif exchange == 'XETRA':
        formatted_ticker = f"{ticker}.DE"  # .V according to the Yahoo Finance
    else:
        formatted_ticker = ticker
    
    # Calculate allocation amount
    # Normalize by the person's total weight to ensure 100% allocation
    allocation_usd = INITIAL_VALUE_USD * (weight / person_weights[name])
    
    portfolio_allocations.append({
        'name': name,
        'ticker': formatted_ticker,
        'weight': weight,
        'normalized_weight': weight / person_weights[name],
        'allocation_usd': allocation_usd
    })

# Create a DataFrame for better visualization
allocations_df = pd.DataFrame(portfolio_allocations)
print("Initial portfolio allocations:")
allocations_df

Initial portfolio allocations:


,name,ticker,weight,normalized_weight,allocation_usd
0,Ivan,VST,0.40,0.400000,40000.000000
1,Ivan,EXPE,0.30,0.300000,30000.000000
2,Ivan,DGX,0.30,0.300000,30000.000000
3,Abhi,ASR,0.40,0.400000,40000.000000
4,Abhi,0700.HK,0.25,0.250000,25000.000000
5,Abhi,SMCI,0.35,0.350000,35000.000000
6,Conor,VLA.PA,0.20,0.200000,20000.000000
7,Conor,RR.L,0.20,0.200000,20000.000000
8,Conor,CTC-A.TO,0.20,0.200000,20000.000000
9,Conor,SHEL,0.20,0.200000,20000.000000


In [97]:
# Visualize the allocations by person
fig = px.treemap(
    allocations_df,
    path=['name', 'ticker'],
    values='allocation_usd',
    title='Portfolio Allocations by Person and Stock',
    color='allocation_usd',
    color_continuous_scale='Viridis',
    hover_data=['normalized_weight']
)

fig.update_layout(template='presentation')
fig.update_traces(texttemplate="%{label}<br>$%{value:,.0f}<br>%{customdata[0]:.1%}", textposition="middle center")
fig.show()

In [98]:
# Let's check if we have data for all tickers
print("Checking data availability for all tickers:")
for ticker in unique_tickers:
    if ticker in prices_df.columns:
        data_count = prices_df[ticker].count()
        print(f"{ticker}: {data_count} data points available")
    else:
        print(f"{ticker}: No data available")

# If 0700.HK is missing, try to download it directly
if '0700.HK' not in prices_df.columns or prices_df['0700.HK'].isna().all():
    print("\nAttempting to download 0700.HK directly...")
    tencent_data = yf.download('0700.HK', start=START_DATE, end=END_DATE)
    
    if not tencent_data.empty:
        # Add to our main dataframe
        prices_df['0700.HK'] = tencent_data['Close']
        
        # Convert HKD to USD
        common_dates = prices_df.index.intersection(exchange_rate.index)
        prices_df.loc[common_dates, '0700.HK'] = prices_df.loc[common_dates, '0700.HK'] * exchange_rate.loc[common_dates]
        print(f"Successfully downloaded and converted 0700.HK prices from HKD to USD")
        
        # Check data again
        data_count = prices_df['0700.HK'].count()
        print(f"0700.HK: {data_count} data points available after direct download")
    else:
        print(f"Error: Could not download data for 0700.HK")

Checking data availability for all tickers:
VST: 2112 data points available
EXPE: 4934 data points available
DGX: 6328 data points available
ASR: 6141 data points available
0700.HK: 5087 data points available
SMCI: 4510 data points available
VLA.PA: 4497 data points available
RR.L: 5370 data points available
CTC-A.TO: 5371 data points available
SHEL: 6328 data points available
CRWD: 1438 data points available
FLUT: 5637 data points available
NVDA: 6328 data points available
SCAN.V: 2933 data points available
MP: 1179 data points available
DBK.DE: 5385 data points available
RTX: 6328 data points available
RHM.DE: 5385 data points available
NVO: 6328 data points available
NVDA: 6328 data points available
OPRA: 1657 data points available


In [99]:
# Calculate the number of shares to buy based on the price at FIXED_DATE
# First, find the closest date in our data to FIXED_DATE
closest_date = prices_df.index[prices_df.index <= FIXED_DATE][-1]
print(f"Using prices from {closest_date} for allocation calculations")

# Get prices on that date
prices_on_date = prices_df.loc[closest_date]

# Calculate shares and actual allocation
portfolio = []

for alloc in portfolio_allocations:
    ticker = alloc['ticker']
    allocation_usd = alloc['allocation_usd']
    
    # Get the price on the fixed date
    if ticker in prices_on_date and not pd.isna(prices_on_date[ticker]):
        price = prices_on_date[ticker]
        shares = allocation_usd / price
        actual_allocation = shares * price
        
        portfolio.append({
            'name': alloc['name'],
            'ticker': ticker,
            'shares': shares,
            'price_at_start': price,
            'allocation_usd': actual_allocation
        })
    else:
        print(f"Warning: No price data available for {ticker} on {closest_date}")

# Create a DataFrame for the portfolio
portfolio_df = pd.DataFrame(portfolio)
print("\nFinal portfolio with share allocations:")
portfolio_df

Using prices from 2025-02-21 00:00:00 for allocation calculations

Final portfolio with share allocations:


,name,ticker,shares,price_at_start,allocation_usd
0,Ivan,VST,265.886728,150.440002,40000.000000
1,Ivan,EXPE,152.431281,196.809998,30000.000000
2,Ivan,DGX,174.175563,172.240005,30000.000000
3,Abhi,ASR,146.246933,273.510010,40000.000000
4,Abhi,0700.HK,375.997095,66.489875,25000.000000
5,Abhi,SMCI,624.219729,56.070000,35000.000000
6,Conor,VLA.PA,5136.623392,3.893608,20000.000000
7,Conor,RR.L,25.817499,774.668369,20000.000000
8,Conor,CTC-A.TO,199.929496,100.035264,20000.000000
9,Conor,SHEL,299.850089,66.699997,20000.000000


In [100]:
# Calculate daily portfolio value from FIXED_DATE onwards
# Filter prices to dates >= FIXED_DATE
prices_since_start = prices_df[prices_df.index >= closest_date]

# Initialize a DataFrame to store portfolio values
portfolio_values = pd.DataFrame(index=prices_since_start.index)

# Calculate daily value for each person
for name in set(p['name'] for p in portfolio):
    person_portfolio = [p for p in portfolio if p['name'] == name]
    daily_values = pd.Series(0, index=prices_since_start.index)
    
    for position in person_portfolio:
        ticker = position['ticker']
        shares = position['shares']
        
        if ticker in prices_since_start.columns:
            # Calculate daily value of this position
            position_value = prices_since_start[ticker] * shares
            daily_values += position_value
    
    portfolio_values[name] = daily_values

# Display the first few rows
print("Daily portfolio values since start date (USD):")
portfolio_values.head()

Daily portfolio values since start date (USD):


,Radu,Abhi,Ivan,Silvia,Conor,Diarbhail
Date,,,,,,
2025-02-21,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
2025-02-24,101637.508446,95822.549402,98756.669369,98083.185079,99892.729090,96795.048702
2025-02-25,102570.990860,91345.711170,97672.905060,96701.810286,98830.195506,96014.795008
2025-02-26,105499.593215,96214.489149,100030.386007,97682.334964,100415.427824,98524.379469
2025-02-27,105316.292941,89508.723790,93950.825337,94151.057490,102018.739635,95531.995196


In [101]:
# Plot the portfolio values over time using Plotly
fig = go.Figure()

# Plot individual portfolios
for name in portfolio_values.columns:
    fig.add_trace(go.Scatter(
        x=portfolio_values.index,
        y=portfolio_values[name],
        name=f"{name}'s Portfolio",
        mode='lines'
    ))

# Add a horizontal line for initial investment
fig.add_shape(
    type="line",
    x0=portfolio_values.index[0],
    y0=INITIAL_VALUE_USD,
    x1=portfolio_values.index[-1],
    y1=INITIAL_VALUE_USD,
    line=dict(color="red", width=2, dash="dash")
)

# Update layout
fig.update_layout(
    title='Portfolio Performance Since Start Date',
    xaxis_title='Date',
    yaxis_title='Portfolio Value (USD)',
    template='presentation',
    hovermode='x unified'
)

# Show the figure
fig.show()

In [102]:
# Calculate percentage change from start date
pct_change = portfolio_values.copy()
for col in pct_change.columns:
    pct_change[col] = (pct_change[col] / pct_change[col].iloc[0] - 1) * 100

# Plot the percentage change
fig = go.Figure()

# Plot individual portfolios
for name in pct_change.columns:
    fig.add_trace(go.Scatter(
        x=pct_change.index,
        y=pct_change[name],
        name=f"{name}'s Portfolio",
        mode='lines'
    ))

# Add a horizontal line at 0%
fig.add_shape(
    type="line",
    x0=pct_change.index[0],
    y0=0,
    x1=pct_change.index[-1],
    y1=0,
    line=dict(color="red", width=2, dash="dash")
)

# Update layout
fig.update_layout(
    title='Portfolio Performance (% Change) Since Start Date',
    xaxis_title='Date',
    yaxis_title='Percentage Change (%)',
    template='presentation',
    hovermode='x unified'
)

# Show the figure
fig.show()

In [103]:
# Calculate performance metrics
initial_value = portfolio_values.iloc[0]
final_value = portfolio_values.iloc[-1]

# Calculate returns
total_return = (final_value / initial_value - 1) * 100

# Calculate annualized returns
days = (portfolio_values.index[-1] - portfolio_values.index[0]).days
years = days / 365.25
annualized_return = ((final_value / initial_value) ** (1 / years) - 1) * 100

# Create a DataFrame for the results
performance_df = pd.DataFrame({
    'Initial Value': initial_value,
    'Final Value': final_value,
    'Total Return (%)': total_return,
    'Annualized Return (%)': annualized_return
})

print("Performance Summary:")
performance_df

Performance Summary:


,Initial Value,Final Value,Total Return (%),Annualized Return (%)
Radu,100000.0,105966.945846,5.966946,1957.578539
Abhi,100000.0,88626.336276,-11.373664,-99.816388
Ivan,100000.0,95828.672250,-4.171328,-89.174197
Silvia,100000.0,97032.056647,-2.967943,-79.238643
Conor,100000.0,100979.345639,0.979346,66.283256
Diarbhail,100000.0,92219.584533,-7.780415,-98.539429


In [104]:
# Create a bar chart to visualize the total returns
performance_data = pd.DataFrame({
    'Portfolio': total_return.index,
    'Total Return (%)': total_return.values
})

# Sort the data to make the visualization more informative (optional)
performance_data = performance_data.sort_values('Total Return (%)', ascending=False)

# Create the figure with increased size
fig = px.bar(
    performance_data,
    x='Portfolio',
    y='Total Return (%)',
    title='Portfolio Returns by Competitor',
    color='Total Return (%)',
    color_continuous_scale=['red', 'yellow', 'green'],
    text='Total Return (%)'
)

# Add a horizontal line at 0%
fig.add_shape(
    type="line",
    x0=-0.5,
    y0=0,
    x1=len(performance_data)-0.5,
    y1=0,
    line=dict(color="black", width=2, dash="dash")
)

# Format the text labels and ensure they're visible
fig.update_traces(
    texttemplate='%{text:.1f}%', 
    textposition='outside',
    textfont=dict(size=14)  # Increase text size
)

# Calculate the y-axis range with extra padding to ensure text is visible
y_min = min(performance_data['Total Return (%)']) * 1.2 if min(performance_data['Total Return (%)']) < 0 else min(performance_data['Total Return (%)']) - 2
y_max = max(performance_data['Total Return (%)']) * 1.2 if max(performance_data['Total Return (%)']) > 0 else max(performance_data['Total Return (%)']) + 2

# Update layout with increased size, margins, and adjusted y-axis
fig.update_layout(
    template='presentation',
    yaxis=dict(
        title='Total Return (%)',
        range=[y_min, y_max]  # Ensure text labels are visible
    ),
    width=900,  # Increase width
    height=600,  # Increase height
    margin=dict(l=80, r=80, t=100, b=80),  # Increase margins
    title=dict(
        text='Portfolio Returns by Competitor',
        font=dict(size=24),  # Larger title
        y=0.95  # Position title higher
    )
)

# Show the figure
fig.show()